In [1]:
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_box_markup as show_span_markup, show_dep_ascii_markup as show_dep_markup
from slovnet import Morph, Syntax
from razdel import sentenize, tokenize

from yargy import Parser, rule, and_, not_, or_
from yargy.interpretation import fact
from yargy.predicates import gram, tag
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline

In [23]:
text = 'Заключенные ИК-2 рассказали, что Навального привезли в колонию. «Приемку этапа» провели строго по правилам внутреннего распорядка и никого не били. Теперь заключенные опасаются, что из-за Навального режим ужесточат, а «жизнь в заключении наладилась».'

In [24]:
navec = Navec.load('./data/navec_news_v1_1B_250K_300d_100q.tar')

ner = NER.load('./data/slovnet_ner_news_v1.tar')
morph = Morph.load('./data/slovnet_morph_news_v1.tar', batch_size=4)
syntax = Syntax.load('./data/slovnet_syntax_news_v1.tar')

ner.navec(navec)
morph.navec(navec)
syntax.navec(navec)

print("load: ok")

load: ok


In [25]:
markup = ner(text)

In [26]:
show_span_markup(markup.text, markup.spans)
markup.spans

[Span(
     start=12,
     stop=16,
     type='ORG'
 ),
 Span(
     start=33,
     stop=43,
     type='PER'
 ),
 Span(
     start=188,
     stop=198,
     type='PER'
 )]

In [27]:
chunk = []
for sent in sentenize(text):
    tokens = [_.text for _ in tokenize(sent.text)]
    chunk.append(tokens)

In [31]:
chunk

[['Заключенные',
  'ИК-2',
  'рассказали',
  ',',
  'что',
  'Навального',
  'привезли',
  'в',
  'колонию',
  '.'],
 ['«',
  'Приемку',
  'этапа',
  '»',
  'провели',
  'строго',
  'по',
  'правилам',
  'внутреннего',
  'распорядка',
  'и',
  'никого',
  'не',
  'били',
  '.'],
 ['Теперь',
  'заключенные',
  'опасаются',
  ',',
  'что',
  'из-за',
  'Навального',
  'режим',
  'ужесточат',
  ',',
  'а',
  '«',
  'жизнь',
  'в',
  'заключении',
  'наладилась',
  '»',
  '.']]

In [32]:
sent = chunk[0]
markup = morph(sent)
for token in markup.tokens:
    print(f'{token.text:>20} {token.tag}')

         Заключенные NOUN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Plur
                ИК-2 PROPN|Foreign=Yes
          рассказали VERB|Aspect=Perf|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act
                   , PUNCT
                 что SCONJ
          Навального PROPN|Animacy=Anim|Case=Acc|Gender=Masc|Number=Sing
            привезли VERB|Aspect=Perf|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act
                   в ADP
             колонию NOUN|Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing
                   . PUNCT


In [34]:
markup = syntax(sent)

In [35]:
markup.tokens[0]

SyntaxToken(
    id='1',
    text='Заключенные',
    head_id='3',
    rel='nsubj'
)

In [36]:
words, deps = [], []
for token in markup.tokens:
    words.append(token.text)
    source = int(token.head_id) - 1
    target = int(token.id) - 1
    if source > 0 and source != target:  # skip root, loops
        deps.append([source, target, token.rel])

In [10]:
show_dep_markup(words, deps)

        ┌► Заключенные nsubj
        │  ИК-2        
┌─┌─────└─ рассказали  
│ │ ┌────► ,           punct
│ │ │ ┌──► что         mark
│ │ │ │ ┌► Навального  obj
│ └►└─└─└─ привезли    ccomp
│   │   ┌► в           case
│   └──►└─ колонию     obl
└────────► .           punct


In [11]:
def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        spans = [_.span for _ in matches]
        show_span_markup(line, spans)

In [12]:
INT = type('INT')
N = gram('NOUN')
ADJF = gram('ADJF')
ABBR = gram('Abbr')

NOUN = and_(
    N,
    not_(ABBR)
)

gnc = gnc_relation()

CONCEPT = fact('Concept', ['text'])

NP = rule(
    ADJF.optional().repeatable().match(gnc),
    NOUN.repeatable().match(gnc)
).interpretation(CONCEPT.text.inflected())

In [13]:
show_matches(NP, text)

In [14]:
parser = Parser(NP)
matches = parser.findall(text)
for match in matches:
    display(match)

Match(
    tokens=[MorphToken(
         value='Навального',
         span=[33, 43),
         type='RU',
         forms=[Form('навальный', Grams(ADJF,gent,masc,sing)),
          Form('навальный', Grams(ADJF,accs,anim,masc,sing)),
          Form('навальный', Grams(ADJF,gent,neut,sing)),
          Form('навальный', Grams(NOUN,Sgtm,Surn,anim,gent,masc,sing)),
          Form('навальный', Grams(NOUN,Sgtm,Surn,accs,anim,masc,sing))]
     )],
    span=[33, 43)
)

Match(
    tokens=[MorphToken(
         value='колонию',
         span=[55, 62),
         type='RU',
         forms=[Form('колония', Grams(NOUN,accs,femn,inan,sing))]
     )],
    span=[55, 62)
)

Match(
    tokens=[MorphToken(
         value='Приемку',
         span=[65, 72),
         type='RU',
         forms=[Form('приёмка', Grams(NOUN,accs,femn,inan,sing))]
     ),
     MorphToken(
         value='этапа',
         span=[73, 78),
         type='RU',
         forms=[Form('этап', Grams(NOUN,gent,inan,masc,sing))]
     )],
    span=[65, 78)
)

Match(
    tokens=[MorphToken(
         value='правилам',
         span=[98, 106),
         type='RU',
         forms=[Form('правило', Grams(NOUN,datv,inan,neut,plur))]
     )],
    span=[98, 106)
)

Match(
    tokens=[MorphToken(
         value='внутреннего',
         span=[107, 118),
         type='RU',
         forms=[Form('внутренний', Grams(ADJF,gent,masc,sing))]
     ),
     MorphToken(
         value='распорядка',
         span=[119, 129),
         type='RU',
         forms=[Form('распорядок', Grams(NOUN,gent,inan,masc,sing))]
     )],
    span=[107, 129)
)

Match(
    tokens=[MorphToken(
         value='из',
         span=[182, 184),
         type='RU',
         forms=[Form('из', Grams(PREP)),
          Form('иза', Grams(NOUN,Name,anim,femn,gent,plur)),
          Form('иза', Grams(NOUN,Name,accs,anim,femn,plur))]
     )],
    span=[182, 184)
)

Match(
    tokens=[MorphToken(
         value='Навального',
         span=[188, 198),
         type='RU',
         forms=[Form('навальный', Grams(ADJF,gent,masc,sing)),
          Form('навальный', Grams(ADJF,accs,anim,masc,sing)),
          Form('навальный', Grams(ADJF,gent,neut,sing)),
          Form('навальный', Grams(NOUN,Sgtm,Surn,anim,gent,masc,sing)),
          Form('навальный', Grams(NOUN,Sgtm,Surn,accs,anim,masc,sing))]
     ),
     MorphToken(
         value='режим',
         span=[199, 204),
         type='RU',
         forms=[Form('режим', Grams(NOUN,inan,masc,nomn,sing)),
          Form('режим', Grams(NOUN,accs,inan,masc,sing))]
     )],
    span=[188, 204)
)

Match(
    tokens=[MorphToken(
         value='жизнь',
         span=[219, 224),
         type='RU',
         forms=[Form('жизнь', Grams(NOUN,accs,femn,inan,sing)),
          Form('жизнь', Grams(NOUN,femn,inan,nomn,sing))]
     )],
    span=[219, 224)
)

Match(
    tokens=[MorphToken(
         value='заключении',
         span=[227, 237),
         type='RU',
         forms=[Form('заключение', Grams(NOUN,inan,loct,neut,sing))]
     )],
    span=[227, 237)
)